In [ ]:
%%capture
%run Vector.ipynb

Here we present algorithms for solving the Kepler and Gauss problems. the two fundamental problems in
two-body mechanics

The *Kepler Problem*, or prediction problem, is as follows: Given the $\mu=GM_{center}$ of the
central body, and the position and velocity of a test particle of negligible mass at 
one time, find the position and velocity at any other time.

The *Gauss Problem*, or targeting problem, is as follows: Given the $\mu$ of the 
central body, the position at which a test particle is now, the target position where
it will be, and the time between the postions, find the velocity of the particle at 
both points such that it will travel from the initial position to the target position
in the given time.

This basically follows the Universal Variable formation found in Bate, Muller, and White
chapters 4 and 5.

# Canonical Units
These algorithms make use of canonical units. Canonical units are *scaled* distance and time units relating to a particular central body, such that $\mu=1$ in that context. The parameter $\mu$ or its powers is almost always used as a multiplicative constant. Therefore in any formula involving $\mu$, setting $\mu=1$ is equivalent to just cancelling it out.

In canonical units, an object in a circular orbit of radius one Distance Unit (DU) has a speed of one DU per Time Unit (TU), and therefore an angular velocity of one radian per TU. 

The length of a DU is arbitrary, but is customarily the radius of the central body for planets and moons, and 1 AU for the Sun. Once the length of a DU is decided and the $\mu$ of the central object is known in some standard system of units (like SI), the duration of a TU is determined.

As we shall see, many of the documented math formulas have an explicit $\mu$ term, but most often in the code, the parameters are translated from standard physical units to canonical units at the top of the function, and translated back at the bottom, so that the hard part in the middle doesn't have `mu` at all.   

In [ ]:
import numpy as np
from collections import namedtuple
import math
import matplotlib.pyplot as plt


def su_to_cu(x,a,mu,L,T,inverse=False):
    """
    Convert standard units to canonical units
    input
    :param x:  measurement to convert, may be scalar or any size or shape of vector
    :param a:  length of canonical distance unit in standard units. Standard length unit is implied by this. 
    :param mu: gravitational constant in standard distance and time units. Standard length unit same as 
               above, standard time unit implied by this.
    :param L: power of length dimension used in x
    :param T: power of time dimension used in x
    :param inverse: convert x from canonical units back to standard units
    :return: a scalar or array the same size as x, in canonical units (or standard if /inv was set)
    """
    #To convert:  Standard Unit to       Canonical Unit          Multiply by
    #Distance     m                      DU                      1/a
    #Time         s                      TU                      sqrt(mu/a**3)
    #For derived units, raise the base unit for each dimension to the power of the dimension needed,
    #then multiply. For example:
    #Speed        m/s                    DU/TU (LL=1,TT=-1)      1/a*1/sqrt(a**3/mu)
    #For inverse conversion, divide instead of multiply
    DU=(1.0/a)**L
    TU=np.sqrt(mu/a**3)**T
    DUTU=DU*TU
    if inverse:
        DUTU=1.0/DUTU
    return x*DUTU

## Example
An object orbiting Earth has a position of:

In [ ]:
r=np.array([[1131340],[-2282343],[6672423]]) #m

and a velocity of:

In [ ]:
v=np.array([[-5643.05],[4303.33],[2428.79]]) #m/s

Convert this to canonical units. Earth radius used as distance unit length:

In [ ]:
Re=6378137 #m

Earth gravitational constant:

In [ ]:
mu=398600.4415e9 #m,s

In [ ]:
print("r_cu=",su_to_cu(r,Re,mu,L=1,T=0))
print("v_cu=",su_to_cu(v,Re,mu,L=1,T=-1))

We are going to solve the Kepler problem over a time of 40min=2400s. How many canonical time units?

In [ ]:
print("%9.7f"%su_to_cu(2400,Re,mu,L=0,T=1)) #should be 2.9746739

Suppose the answer is:

In [ ]:
rt=np.array([[-0.6616125],[ 0.6840739],[-0.6206809]])
vt=np.array([[ 0.4667380],[-0.2424455],[-0.7732126]])

What is this in SI units?

In [ ]:
print("rt_su=",su_to_cu(rt,Re,mu,L=1,T=0,inverse=True)) #should be -4219855.2       4363117.1      -3958787.8
print("vt_su=",su_to_cu(vt,Re,mu,L=1,T=-1,inverse=True)) #should be  3689.7346      -1916.6203      -6112.5284

# Orbital elements
Orbital elements are surprisingly not used too much -- mostly for output for feeble human minds to grasp. In this code, we are careful to make it so that it can handle stacks of input vectors. 

Many of these elements are just brute-forced from angles between vectors, using the appropriate dot products or cross products. However, one of the more interesting concepts that we see below is the *eccentricity vector* $\vec{e}$. This vector points from the center towards the periapse, and its length is equal to the length of the eccentricity of the orbit. For a circular orbit, $|\vec{e}|=0$ and therefore the vector has no direction, which is to be expected for an orbit with no well-defined periapse.

The formula for the eccentricity vector is not derived here, just given as a recipe, but it is something that falls quite naturally out of the calculus that proves Kepler's second law (all orbits trace conic sections).

$$\begin{eqnarray*}
\vec{e}_1&=&\left(v^2 - \frac{1}{r}\right)\vec{r} \\
\vec{e}_2&=&\left(\vec{r}\cdot\vec{v}\right)\vec{v} \\
\vec{e}  &=&\frac{1}{\mu}\left[\vec{e}_1-\vec{e}_2\right] \\
         &=&\frac{1}{\mu}\left[\left(v^2 - \frac{1}{r}\right)\vec{r}-\left(\vec{r}\cdot\vec{v}\right)\vec{v}\right]
\end{eqnarray*}$$

Without discussing special cases (the code handles all known special cases), let's look at how the other elements are derived:

* The orbit pole is a vector from the center, perpendicular to the orbit plane. It is in the same direction as the angular momentum vector $\vec{h}=\vec{r}\times\vec{v}$.
* The orbit plane crosses the equatorial plane along a line which is perpendicular to both the orbit pole and the equatorial pole $\vec{z}=\begin{bmatrix}0 & 0 & 1\end{bmatrix}^T$, so we define the ascending node vector as $\vec{n}=\vec{z}\times\vec{h}$
* The inclination is found from the angle between the orbit pole and equatorial pole
* The longitude of ascending node is found from the coordinates of the ascending node vector
* The argument of periapse is found from the angle between the ascending node vector and eccentricity vector
* The true anomaly is found from the angle between the eccentricity vector and the current position vector
* The semimajor axis is found from the orbit energy, which itself is the sum of the kinetic energy determined from the speed, and the potential energy determined from the radial distacne to the center.

In [ ]:
Elorb=namedtuple('elorb',['p','a','e','i','an','ap','ta','tp','rp','MM','n','t'])
def elorb(rv, vv=None, l_DU=None, mu=None, t0=None):
    """
    Given state vector, calculate orbital elements.
    :param r: position vector, in distance units implied by mu, any inertial frame is fine,
              but center of attraction is assumed to be at the origin of the frame
    :param v: inertial velocity vector, distance and time units implied by mu, must be in
              same frame as r. May be None, in which case r is treated as a complete state vector.
    :param l_du: Optional length of a distance unit, used for conversion to canonical units internally. If not passed,
                 assume that input is already in canonical units
    :param mu: gravity parameter, implies distance and time units. Required if l_du is passed
    :param t0: optional time of given state vector, in time units implied by mu. Pass this if you want
               absolute time for periapse passage.
    :return: a named tuple
        p:  semi-parameter, distance from focus to orbit at TA=+-90deg, always positive for any eccentricity
        a:  semimajor axis
        e:  eccentricity
        i:  inclination
        an: longitude of ascending node, angle between x axis and line of intersection between orbit plane
            and xy plane
        ap: argument of periapse, angle between xy plane and periapse along orbit plane
        ta: true anomaly, angle between periapse and object
        tp: time to next periapse. If t0 is passed, then this is an absolute time of periapse passage. If not,
            this is a time interval from the implied epoch of the given state vector. Will be before the epoch
            if and only if there is only one periapse (parabolic or hyperbolic) and it is in the past.
        rp: radius of periapse
    If no l_du is passed, canonical units are assumed -- all lengths passed and returned are canonical. If
    an l_du *is* passed, standard units are converted internally to distance units, and results are converted
    back to standard units upon return. All angles are always returned as radians.
    """
    if vv is None:
        #Passed a state vector, split it in half and re-call ourselves
        if vindex(rv)==0:
            vv=rv[vncomp(rv)//2:,...]
            rv=rv[:vncomp(rv)//2,...]
        else:
            vv = rv[...,vncomp(rv) // 2:, :]
            rv = rv[...,:vncomp(rv) // 2, :]
        return elorb(rv,vv,l_DU,mu,t0)
    if l_DU is not None:
        rv = su_to_cu(rv, l_DU, mu, 1, 0)
        vv = su_to_cu(vv, l_DU, mu, 1, -1)
    r = vlength(rv, array=True)
    v = vlength(vv, array=True)
    #Angular momentum vector, same direction as orbit pole
    hv = vcross(rv, vv)
    h = vlength(hv, array=True)
    #Node vector, points in direction of ascending node
    nv = vcross(np.array([0, 0, 1]), hv)
    n = vlength(nv, array=True)
    ev1 = (v ** 2 - 1.0 / r) * rv
    ev2 = vdot(rv, vv, array=True) * vv
    ev = ev1 - ev2
    e = vlength(ev, array=True)

    xi = v ** 2 / 2.0 - 1 / r
    a = xi * 0
    p = a * 0
    # non-parabolic case
    wnp = np.where(e != 1.0)
    a[wnp] = (-1 / (2 * xi[wnp]))
    p[wnp] = a[wnp] * (1 - e[wnp] ** 2)
    # parabolic case
    wp = np.where(e == 1.0)
    a[wp] = float('inf')
    p[wp] = h[wp] ** 2
    (rx, ry, rz) = vcomp(rv,3)
    (hx, hy, hz) = vcomp(hv,3)
    (nx, ny, nz) = vcomp(nv,3)
    i = np.arccos(hz / h)
    #Trap nx/n is 0/0, which happens in an equatorial orbit. In such a case, ascending node is undefined.
    #It could equally well be said to be anywhere, so we will arbitrarily force these cases to zero,
    #which results in argument of periapse being equivalent to longitude of periapse
    import numpy.testing
    with numpy.testing.suppress_warnings() as sup:
        sup.filter(RuntimeWarning)
        an = np.arccos(nx / n)
    an[np.where(np.isnan(an))]=0

    wy=np.where(ny<0)
    an[wy] = 2 * np.pi - an[wy]
    #Trap vangle(ev,nv) for zero-length ev or nv, which happens in a circular or equatorial orbit.
    #In such a case, argument of periapse is undefined, because there is no periapse (circular orbit)
    #or there is no node (equatorial orbit). We will arbitrarily force this to zero, just to arbitrarily
    #force the orbit to have *some* reference point for true anomaly
    with numpy.testing.suppress_warnings() as sup:
        sup.filter(RuntimeWarning)
        ap = vangle(ev, nv,True)
    ap[np.where(np.isnan(ap))]=0
    (ex, ey, ez) = vcomp(ev,3)
    wz=np.where(ez<0)
    ap[wz] = 2 * np.pi - ap[wz]
    #Trap vangle(ev,rv) for zero-length ev, which happens in a circular orbit. In such a case we want the angle
    #from the node if there is one, or from the longitude reference if the plane is
    with numpy.testing.suppress_warnings() as sup:
        sup.filter(RuntimeWarning)
        #First guess - just calculate the angle between ev and rv. This will be correct
        #when ev is defined (non-circular orbit) and object is going uphill.
        ta = vangle(ev, rv, True)
        #Correct the ta when the orbit is going downhill
        w = np.where(vdot(rv, vv, True) < 0)
        ta[w] = 2 * np.pi - ta[w]
        #If it fails to have a TA at this point, it is because it is a circular
        #orbit (ev is zero vector) and vangle failed. Use the node as ta reference.
        #Don't have to worry about uphill or downhill - instead worry about above
        #or below plane.
        w=np.where(np.isnan(ta))
        #Check for both circular orbits and below plane
        ww=np.where(np.logical_and(np.isnan(ta),rz<0))
        ta[w]=vangle(rv,nv,True)[w]
        #correct for circular below plane case
        ta[ww]=2*np.pi-ta[ww]
        #If no answer yet, it's because the orbit is both circular AND equatorial.
        #In this case, use longitude reference as ta reference by using atan2 on
        #x and y components of rv
        w=np.where(np.isnan(ta))
        ta[w]=np.arctan2(ry,rx)[w]
    EE=i*0
    MM=i*0
    n=i*0
    rp=i*0
    #Parabolic case
    wp=np.where(np.logical_not(np.isfinite(a)))
    EE[wp] = np.tan(ta[wp]) / 2
    MM[wp] = EE[wp] ** 3 / 3 + EE[wp]
    n[wp] = np.sqrt(2 / (p[wp] ** 3))
    rp[wp] = p[wp] / 2
    #Elliptical case
    we=np.where(np.logical_and(np.isfinite(a),a>0))
    EE[we] = 2 * np.arctan(np.sqrt((1 - e[we]) / (1 + e[we])) * np.tan(ta[we] / 2))
    MM[we] = EE[we] - e[we] * np.sin(EE[we])
    n[we] = np.sqrt(1 / (a[we] ** 3))
    rp[we] = a[we] * (1 - e[we])
    #Hyperbolic case
    wh=np.where(np.logical_and(np.isfinite(a),a<0))
    EE[wh] = np.arcsinh(np.sin(ta[wh]) * np.sqrt(e[wh] ** 2 - 1) / (1 + e[wh] * np.cos(ta[wh])))
    MM[wh] = e[wh] * np.sinh(EE[wh]) - EE[wh]
    n[wh] = np.sqrt(-1 / (a[wh] ** 3))
    rp[wh] = a[wh] * (1 - e[wh])
    tp = -MM / n
    #Trap e>=1, which will have a=inf or a<0. The former will result in t=inf, the latter in t=NaN, both
    #of which are appropriate.
    with numpy.testing.suppress_warnings() as sup:
        sup.filter(RuntimeWarning)
        t = 2 * math.pi * np.sqrt(a ** 3)
    if l_DU is not None:
        p = su_to_cu(p, l_DU, mu, 1, 0, inverse=True)
        a = su_to_cu(a, l_DU, mu, 1, 0, inverse=True)
        tp = su_to_cu(tp, l_DU, mu, 0, 1, inverse=True)
        if t0 is not None:
            tp += t0
        rp = su_to_cu(rp, l_DU, mu, 1, 0, inverse=True)
        # n = su_to_cu(n, l_DU, mu, 0, -1, inverse=True)
        t = su_to_cu(t, l_DU, mu, 0, 1, inverse=True)
    if p.size==1:
        p=p[0]
        a=a[0]
        e=e[0]
        i=i[0]
        an=an[0]
        ap=ap[0]
        ta=ta[0]
        tp=tp[0]
        rp=rp[0]
        MM=MM[0]
        n=n[0]
        t=t[0]
    return Elorb(p=p, a=a, e=e, i=i, an=an, ap=ap, ta=ta, tp=tp, rp=rp, MM=MM, n=n, t=t)


Above, we were using test data for the Kepler problem. The unit converter doesn't care about it, but let's see what kind of orbit it is working with. What are the orbital elements of the given position?

In [ ]:
print("r_su=",r)
print("v_su=",v)
el=elorb(r,v,Re,mu)
print(el)
print("i(deg)=",np.degrees(el.i))
print("hp=",el.rp-Re)

This orbit is nearly circular with a periapse altitude of about 764km. The orbit inclination is near that needed for a sun-synchronous orbit.

While it doesn't matter to our unit converter if the answer to the Kepler problem is correct, it is interesting to check. If the answer to the Kepler problem above is correct, then most of these elements should be the same for the answer. Let's see if that is true. The given answer is in canonical units, convert it to standard units first so that the length units are comparable:

In [ ]:
rt_su=su_to_cu(rt,Re,mu,L= 1,T= 0,inverse=True)
vt_su=su_to_cu(vt,Re,mu,L= 1,T=-1,inverse=True)
print("rt_su=",rt_su)
print("vt_su=",vt_su)
el=elorb(rt_su,vt_su,Re,mu)
print(el)


The orbit elements do match to sufficient precision, so we accept that this is an answer to the Kepler problem. 

Now we test stacks of arrays, with:

$$\begin{eqnarray*}
[\mathbf{r}]&=&\begin{bmatrix}\vec{r}_0 & \vec{r}_1\end{bmatrix} \\
     &=&\begin{bmatrix}1 & 2 \\ 0 & 0 \\ 0 & 0\end{bmatrix}\\
[\mathbf{v}]&=&\begin{bmatrix}\vec{v}_0 & \vec{v}_1\end{bmatrix} \\
     &=&\begin{bmatrix}0 & 0 \\ 1 & 0 \\ 0.1 & 0.5\end{bmatrix}\\
\end{eqnarray*}$$

Both of these orbits are elliptical and inclined, so should have all elements well-defined:


In [ ]:
rv=np.array([[1,2],[0,0],[0,0]])
vv=np.array([[0,0],[1,0],[0.1,0.5]])
print(elorb(rv,vv))


Now for a few corner cases:

In [ ]:
#1D numpy array instead of column vector. Circular equatorial orbit. 
#Currently fails because it handles 0D arrays at a point and you can't index a 0D array.
#print(elorb(np.array([1,0,0]),np.array([0,1,0]))) 

In [ ]:
#Column vector circular equatorial orbit. Should have well-defined orbital elements
print(elorb(np.array([[1],[0],[0]]),np.array([[0],[1],[0]]))) 

In [ ]:
#Circular polar orbit
print(elorb(np.array([[1],[0],[0]]),np.array([[0],[0],[1]]))) 

In [ ]:
#Same orbit, but with object over the pole instead of equator. Should have nonzero ta
print(elorb(np.array([[0],[0],[1]]),np.array([[-1],[0],[0]]))) 

In [ ]:
#2D case
print(elorb(np.array([[1],[0]]),np.array([[0],[1]])))

In [ ]:
#2D case as state vector
print(elorb(np.array([[1],[0],[0],[1]])))

In [ ]:
#Circular, elliptical, parabolic, hyperbolic orbits. 
#Choose the parabolic case carefully, so that it is exactly parabolic even in the face of floating-point error.
#The escape velocity 2DU out is exactly $v_esc=\sqrt{\frac{2}{r}}=1$DU/TU.
print(elorb(np.array([[1,1,2,1],[0,0,0,0],[0,0,0,0]]),np.array([[0,0,0,0],[0,0,0,0],[1,1.2,1,2]]))) 

# Kepler Problem
The following is presented without too much comment -- see BMW section 4.3-4.5 for details. The solution to both the Kepler and Gauss problems are universal, meaning that it works with any orbit shape -- elliptical, parabolic, or hyperbolic. Therefore we call this the *universal variable* solution. It depends on finding a value for the variable $x$ which does the job of the mean anomaly for any shape of orbit, rather than specific cases as is often presented in textbooks. Since this variable $x$ works for any shape, we call it the titular universal variable.

First, we define the $C$ and $S$ functions, which are required for both the Kepler and Gauss problems:

$$\begin{eqnarray}
C(z)&=&\begin{cases}
1-\frac{\cos\sqrt{z}}{z}, & z>0 \\
\frac{1}{2}, & z=0 \\
1-\frac{\cosh\sqrt{-z}}{z}, & z<0 \\
\end{cases}\\
 &=&\frac{1}{2!}-\frac{z}{4!}+\frac{z^2}{6!}-\frac{z^3}{8!}+\dots\\
 &=&\sum_{k=0}^\infty \frac{(-z)^k}{(2k+2)!}\\
S(z)&=&\begin{cases}
\frac{\sqrt{z}-\sin\sqrt{z}}{z^{3/2}}, & z>0 \\
\frac{1}{6}, & z=0 \\
\frac{\sinh\sqrt{-z}-\sqrt{-z}}{(-z)^{3/2}} & z<0 \\
\end{cases} \\
 &=&\frac{1}{3!}-\frac{z}{5!}+\frac{z^2}{7!}-\frac{z^3}{9!}+\dots\\
 &=&\sum_{k=0}^\infty \frac{(-z)^k}{(2k+3)!}\\
\end{eqnarray}$$

Here we use the closed form, even though it has separate cases, to take advantage of the library transcendental functions. These are most likely computed with series themselves, but they are computed in machine language so that we don't have to loop in the interpreter and don't have to make any convergence checks.

In [ ]:
def C(z,array=True):
    """
    Calculate the Universal Variable C(z) function
    """
    if np.isscalar(z):
        z=np.array([z])
    result=np.zeros(z.shape)
    wgz=np.where(z>0)
    result[wgz]=(1-np.cos(np.sqrt(z[wgz])))/z[wgz]
    wez=np.where(z==0)
    result[wez]=0.5
    wlz=np.where(z<0)
    result[wlz]=(1-np.cosh(np.sqrt(-z[wlz])))/z[wlz]
    if ~array and result.size==1:
        result=result[0]
    return result

def S(z,array=True):
    """
    Calculate the Universal Variable S(z) function
    """
    if np.isscalar(z):
        z=np.array([z])
    result=np.zeros(z.shape)
    wgz=np.where(z>0)
    sz=np.sqrt(z[wgz])
    result[wgz]=(sz-np.sin(sz))/sz**3
    wez=np.where(z==0)
    result[wez]=1.0/6.0
    wlz=np.where(z<0)
    sz=np.sqrt(-z[wlz])
    result[wlz]=(np.sinh(sz)-sz)/np.sqrt((-z[wlz])**3)
    if ~array and result.size==1:
        result=result[0]
    return result

plt.figure("C and S functions")
z=np.arange(-(4*np.pi)**2,(4*np.pi)**2,0.01)
plt.plot(z,np.log10(C(z)),label='log(C(z))')
plt.plot(z,np.log10(S(z)),label='log(S(z))')
plt.legend()
plt.show()

We see that $C(z)$ touches zero (as it turns out, at $z=(2n\pi)^2$ for all integer $n$) while $S(z)$ tends asymtotically towards zero.

Next the Kepler solver itself. It works by finding by Newton's method, a value of $x$ such that the time covered by the change in $x$ from the current position to the new position, matches the given time. Here, we show the algorithm, but not discuss the derivation. To make the intermediate formulas easier, we use canonical units, so all mentions of $\mu$ go to 1 and cancel out. During the iteration, We calculate the value $z$ from the value $x$ in order to feed the $C(z)$ and $S(z)$ functions:

$$\begin{eqnarray*}
z&=&\frac{x^2}{a} \\
 &=&\alpha x^2 \\
\end{eqnarray*}$$

where $a$ is the orbit semimajor axis, calculated from the orbit energy, itself calculable from the initial position $\vec{r}_0$ and velocity $\vec{v}_0$. The method is the same used above in `elorb()`. We use $\alpha=\frac{1}{a}$ because while $a$ is infinite for parabolic orbits, $\alpha$ is finite for all orbit shapes.

We then calculate intermediate value $r$ (I think this is the radial distance given a guesstimated universal variable $x$):

$$r=x^2C(z) + \left(\vec{r}_0\cdot\vec{v}_0\right)x\left(1 - zS(z)\right) + r0\left(1-zC(z)\right) $$

and the time-of-flight $t$ to the guesstimated value of $x$:

$$t^+=x^3S(z) + \left(\vec{r}_0\cdot\vec{v}_0\right)x^2C(z) + r_0x(1 - zS(z))$$

Next a new estimate of $x$ (shown as $x^+$ below) is calculated:

$$x^+=x + \frac{t - t^+}{r}$$

We iterate until the change in $x$, $\Delta x=x-x^+$ is small enough.

Then from that value of $x$ and it's associated $z$, we calculates intermediate variables $f$ and $g$, and their derivatives with respect to time $\dot{f}$ and $\dot{g}$:

$$\begin{eqnarray*}
f&=&1 - \frac{x^2C(z)}{r_0} \\
g&=&t - x^3S(z) \\
\dot{f}&=&\frac{x(zS(z)-1)}{rr_0} \\
\dot{g}&=&1 - \frac{x^2C(z)}{r} 
\end{eqnarray*}$$

Finally the propagated position is found using the following formulas:

$$\begin{eqnarray*}
\vec{r}&=&f\vec{r}_0+g\vec{v}_0 \\
\vec{v}&=&\dot{f}\vec{r}_0+\dot{g}\vec{v}_0 \\
\end{eqnarray*}$$

In [ ]:
def kepler(rv0, vv0, t, l_DU=None, mu=None, eps=1e-9):
    """
    Solve the Kepler problem -- Given a state vector and time interval,
    calculate the state after the time interval elapses
    :param rv0: position vector relative to central body, may be a stack of vectors,
                but must be at least a column vector
    :param vv0: velocity vector relative to central body, must be compatible by
                broadcast with rv0, usually the same shape. If None, then treat rv0 
                as a full (stack of) state vector(s) and return a (stack of) propagated
                state vector(s) as a result.
    :param t:   time interval from given state to requested state - may be negative.
                May be a stack of times, but must be compatible by broadcast with rv0 and vv0
    :param l_DU: [optional] used to convert rv0 and vv0 to canonical units. Length of
                 distance canonical unit in standard units implied by rv0. If not set,
                 input state and time are already presumed to be in canonical units
    :param mu:   [required if l_DU is set] used to convert to canonical units. Gravitational
                 constant in standard units, where distance units are implied by rv0 and
                 time units by vv0.
    :param eps:  loop termination criterion, default to 1e-9
    :return: A tuple:
      0 - Position vector after passage of time t, in same units as rv0
      1 - Velocity vector after passage of time t, in same units as vv0

    This has been carefully organized to act on stacks, using the numpy broadcasting conventions.
    * If you pass a scalar time and only pass a single position and velocity, it is preferred for
      them to be column vectors. This is the preferred way to propagate a single state vector to
      a single new time. Row vectors are untested.
    * You may pass a stack of position and velocity vectors. The normal way to do this is to
      pass a 1D stack (a 2D array, with size (n_comp,n_stack)) of each, with n_stack being the
      same for both stacks. If you pass a 1D stack of inputs (numpy shape (3,n)) and a single time,
      you will get a 1D result of the same shape. This is the preferred way to propagate a state
      vector point cloud to a single new time.
    * You may pass a stack of times, but it must be broadcast compatible with the input vectors. For
      instance, if the input is column vectors with shape (3,1), pass a 1D array of times of size (n,)
      to get an output of 1D vector stacks of size (3,n). This is the preferred way to get a single
      trajectory sampled at multiple time points.
    * You may pass both a stack of times and a stack of vectors, but be careful. If you pass a stack of
      vectors of size (3,n) and a 1D stack of times (n,), you will get each vector propagated to the
      corresponding time point. If you pass vectors (3,n) and times (m,) where m!=n, it will fail to
      broadcast. However, if you pass vectors (3,n) and times (m,1,1) you will get a 2D stack (m,3,n)
      of return results. This propagates all the state vectors in a point cloud simultaneously to all
      given times and result in multiple trajectories. The trajectory of point k (identified by rv0[:,k]
      and vv0[:,k]) will be in the results rv_t[:,:,k].T and vv_t[:,:,k].T. The point cloud at time i
      (identified by t[i,0,0]) will be in the results rv_t[i,:,:] and vv_t[i,:,:].
    * Higher-order stacks may be possible, but are untested. In general, arrange your t stack such that
      it has 1-indexes wherever the vectors have non-1 indexes.
    * Vectors of any dimensionality are supported -- if you pass 2D rv0 and vv0, you will
      get 2D results. Effectively this is treating the z coordinate as zero. If you pass a 4D or higher
      rv0 and vv0, it will work, but is not guaranteed to be physically meaningful -- I think that
      the inverse square law is an emergent property of 3D space, and 4D Euclidean space is likely
      to have an inverse cube law and completely different trajectories.
    """
    # if t==0.0:
    #    #Shortcut if we ask for zero time interval
    #    return (rv0,vv0)
    if vv0 is None:
        #Passed a state vector, split it in half, re-call ourselves, then tape the results together
        if vindex(rv0)==0:
            vv0=rv0[vncomp(rv0)//2:,...]
            rv0=rv0[:vncomp(rv0)//2,...]
        else:
            vv0 = rv0[...,vncomp(rv0) // 2:, :]
            rv0 = rv0[...,:vncomp(rv0) // 2, :]
        (rvt,vvt)=kepler(rv0,vv0,t,l_DU,mu,eps)
        return np.concatenate((rvt,vvt),axis=vindex(rvt))
    tau = np.pi * 2.0
    if l_DU is not None:
        rv0 = su_to_cu(rv0, l_DU, mu, 1, 0)
        vv0 = su_to_cu(vv0, l_DU, mu, 1, -1)
        t = su_to_cu(t, l_DU, mu, 0, 1)
    if np.isscalar(t):
        t = np.array([t])
    r0 = vlength(rv0, True)
    v0 = vlength(vv0, True)

    r0dv0 = vdot(rv0, vv0, True)

    # Determine specific energy, and thereby the orbit shape
    E = v0 ** 2 / 2 - 1 / r0
    alpha = -2 * E  # alpha is 1/a, reciprocal of semimajor axis (in case of parabola. a can be infinite, but never zero)
    x0 = np.zeros((t * alpha).shape)

    # Starting guess for x. To handle arrays, we calculate the starting guess
    # for all three cases, but only keep the correct one for each instance.
    # It might be possible to optimize, but not going to yet.
    we = np.where(np.ones(t.shape) * alpha > 0)  # elliptical
    x0[we] = (t * alpha)[we]
    wp = np.where(np.ones(t.shape) * alpha == 0)  # parabolic (this will never really happen)
    hv = vcross(rv0, vv0)
    p = vdot(hv, hv, True)
    # acot(x)=tau/4-atan(x)
    s = (tau / 4.0 - np.arctan(3.0 * t * np.sqrt(1.0 / p ** 2))) / 2.0
    w = np.arctan(np.tan(s) ** (1.0 / 3.0))
    x0[wp] = (np.sqrt(p) / np.tan(2 * w))[wp]  # cot(x)=1/tan(x)
    wh = np.where(np.ones(t.shape) * alpha < 0)  # hyperbolic
    sa = np.sqrt(-1.0 / alpha)
    st = np.sign(t)
    st[np.where(st==0)]=1
    x0_a = st * sa
    x0_n = -2 * alpha * t
    x0_d = r0dv0 + st * sa * (1 - r0 * alpha)
    x0[wh] = (x0_a * np.log(x0_n / x0_d))[wh]
    #Patch for t==0, needed only for hyperbolic case, but done for all cases
    #x0[np.where(t==0)]=0

    #This is the reverse of my normal 'while not done:' loop, for reasons explained below
    cont= True
    xn = x0
    while cont:
        z = xn ** 2 * alpha
        CC = C(z)
        SS = S(z)
        r = xn ** 2 * CC + r0dv0 * xn * (1 - z * SS) + r0 * (1.0 - z * CC)
        tn = xn ** 3 * SS + r0dv0 * xn ** 2 * CC + r0 * xn * (1.0 - z * SS)
        xnp1 = xn + (t - tn) / r
        #Be careful with this one -- we don't want to continue on an NaN, so
        #continue if any result is greater than the threshold (NaN compared in any way is false)
        #rather than stop if all are less than threshold (NaN would never pass that test)
        cont = np.any(np.abs(xn - xnp1) > eps)
        xn = xnp1
    x = xn
    f = 1 - x ** 2 * CC / r0
    f[np.where(t==0)]=1 #This and related statements below are the special case for t==0
    g = t - x ** 3 * SS
    g[np.where(t==0)]=0
    fdot = x * (z * SS - 1) / (r * r0)
    fdot[np.where(t==0)]=0
    gdot = 1 - x ** 2 * CC / r
    gdot[np.where(t==0)]=1
    rv_t = f * rv0 + g * vv0
    vv_t = fdot * rv0 + gdot * vv0
    if l_DU is not None:
        rv_t = su_to_cu(rv_t, l_DU, mu, 1, 0, inverse=True)
        vv_t = su_to_cu(vv_t, l_DU, mu, 1, -1, inverse=True)
    return (rv_t, vv_t)


We will use the same test data as above, but this time in canonical unit form:

In [ ]:
def test_kepler():
    #From canonical values in Vallado, example 2-4, p102-103
    r0_cu=np.array([ 0.17738,-0.35784,1.04614])
    v0_cu=np.array([-0.71383, 0.54436,0.30723])
    r1_cu_standard=np.array([-0.6616125, +0.6840739, -0.6206809])
    v1_cu_standard=np.array([ 0.4667380, -0.2424455, -0.7732126])
    dt=2.974674
    (r1_cu,v1_cu)=kepler(r0_cu,v0_cu,dt)
    print("test_kepler() Calculated: ",r1_cu,v1_cu)
    print("test_kepler() Documented: ",r1_cu_standard,v1_cu_standard)
    diffr=np.sqrt(np.sum((r1_cu-r1_cu_standard)**2))
    diffv=np.sqrt(np.sum((v1_cu-v1_cu_standard)**2))
    print("test_kepler() difference: ",diffr,diffv)
    assert(diffr<3e-7 and diffv<3e-7)

In [ ]:
def plot_kepler():
    r0_cu=np.array([[ 0.17738],[-0.35784],[ 1.04614]])
    v0_cu=np.array([[-0.71383],[ 0.54436],[ 0.30723]])
    print("plot_kepler elorb: ",elorb(r0_cu, v0_cu))
    r_cu=np.zeros([3,50])
    for i,dt in enumerate(np.linspace(0,2.974674)):
        (r1_cu,v1_cu)=kepler(r0_cu,v0_cu,dt)
        r_cu[:,i]=r1_cu.ravel()
    plt.subplot(221)
    plt.plot(r_cu[0,:],r_cu[1,:])
    plt.axis('equal')
    plt.subplot(222)
    plt.plot(r_cu[0,:],r_cu[2,:])
    plt.axis('equal')
    plt.subplot(223)
    plt.plot(r_cu[1,:],r_cu[2,:])
    plt.axis('equal')
    plt.show()

In [ ]:
test_kepler()
plt.figure("plot_kepler")
plot_kepler()

In [ ]:
def plot_kepler_tarray():
    r0_cu=np.array([[ 0.17738],
                    [-0.35784],
                    [ 1.04614]])
    v0_cu=np.array([[-0.71383],
                    [ 0.54436],
                    [ 0.30723]])
    print("plot_kepler elorb: ",elorb(r0_cu, v0_cu))
    t=np.linspace(0,2.974674)
    print(t.shape)
    (r_cu,v_cu)=kepler(r0_cu,v0_cu,t)
    print(r_cu.shape)
    plt.subplot(221)
    plt.plot(r_cu[0,:],r_cu[1,:])
    plt.axis('equal')
    plt.subplot(222)
    plt.plot(r_cu[0,:],r_cu[2,:])
    plt.axis('equal')
    plt.subplot(223)
    plt.plot(r_cu[1,:],r_cu[2,:])
    plt.axis('equal')
    plt.show()

In [ ]:
test_kepler()
plt.figure("plot_kepler")
plot_kepler_tarray()

# Gauss Problem
I have seen this referred to as both the Gauss problem (solved by Gauss but in terms of three observations of sky coordinates, rather than two position vectors) and the Lambert problem. BMW calls it the Gauss problem, while Vallado and much of modern software (including the flight software of the Space Shuttle) calls it the Lambert problem, or Lambert targeting. Since this is based off of an implementation of the BMW algorithm, we call it Gauss.

In [ ]:
def gauss(rv1,rv2,t,Type=-1,l_DU=None,mu=None,eps=1e-9):
    def FindTTrialCore(A,SS,X,Y):
        return (X**3)*SS+A*np.sqrt(Y)
    def FindTTrial(A,r1,r2,Z):
        SS=S(Z)
        CC=C(Z)
        if CC==0:
            return float('inf')
        Y=r1+r2-A*(1-Z*SS)/np.sqrt(CC)
        X=np.sqrt(Y/CC)
        return FindTTrialCore(A,SS,X,Y)
    def FindZLo(A,r1,r2):
        eps=1e-9
        #Find the Z which results in a Y of exactly zero, by bisection
        Zhi=0.0;
        Y=1.0;
        Zlo=-1.0;

        while Y>0:
            Zlo*=2.0;
            Y=r1+r2-A*(1-Zlo*S(Zlo))/np.sqrt(C(Zlo))
  
        while True: #Emulate a repeat/until loop
            Z=(Zlo+Zhi)/2
            Y=r1+r2-A*(1-Z*S(Z))/np.sqrt(C(Z))
            if Y*Zlo>0:
                Zlo=Z
            else:
                Zhi=Z
            if np.abs(Zlo-Zhi)<eps: #repeat until this condition is true
                break
        return Z+1e-5

    def FindZLo2(A,r1,r2,T):
        #Find the Z which results in a TTrial of less than T
        Z=-1.0
        TTrial=FindTTrial(A,r1,r2,Z)-T
        while TTrial>0:
            Z*=2
            TTrial=FindTTrial(A,r1,r2,Z)-T
        return Z

    tau=2*np.pi
    if l_DU is not None:
        rv1=su_to_cu(rv1,l_DU,mu,1,0)
        rv2=su_to_cu(rv2,l_DU,mu,1,0)
        t  =su_to_cu(t  ,l_DU,mu,0,1)

    if(Type<0):
        pole=vcross(rv1,rv2)
        if(pole[2])>0:
            #prograde is short way
            Type=(-Type-1)*2+1
        else:
            #prograde is long way way
            Type=(-Type-1)*2+2

    if t<0:
        raise ValueError("Time to intercept is negative. Time travel is not allowed in this universe!")
    r1=vlength(rv1)
    r2=vlength(rv2)
    r1dr2=vdot(rv1,rv2);
    DeltaNu=vangle(rv1,rv2);
    Revs=(Type-1)/2;
    #short-way and long-way are reversed for odd-numbers of complete revs
    if ((Revs%2)==1) ^ ((Type%2)==1):
        #Short way
        DM=1.0
    else:
        #Long way
        DM=-1.0
        DeltaNu=tau-DeltaNu
    if Revs>0:
        minA=r1/2.0
        minT=Revs*np.sqrt(tau*minA**3)
        if minT>t:
            raise ValueError("Can't do it! Minimum trip time for %d revs is %fTU, more than requested %fTU" %
                             (Revs,minT,t))
    A=DM*np.sqrt(r1*r2*(1+np.cos(DeltaNu)))
    if(Revs<1):
        #less than one rev
        if Type==1:
            Zlo=FindZLo(A,r1,r2)
        else:
            Zlo=FindZLo2(A,r1,r2,t)
        Zhi=tau**2;
    else:
        #more than one rev
        #Use Zeno's method
        Zlo=((2*Revs+1)*tau/2.0)^2# Z that gives the lowest TIME, not necessarily lowest Z
        #Zbound is the value of Z which gives an infinite T
        if (Type%2)==1:
            Zbound=(Revs*tau)**2
        else:
            Zbound=((Revs+1)*tau)**2
        Zhi=(Zbound+Zlo)/2.0 #Z that gives the highest TIME, not necessarily highest Z
        while True: #emulate repeat/until 
            Thi=FindTTrial(A,r1,r2,Zhi)
            Zhi=(Zbound+Zhi)/2; #Split the difference between current Zhi and bound
            if Thi>=t:
                break

    #Solve it by bisection
    tnlo=FindTTrial(A,r1,r2,Zlo)
    tnhi=FindTTrial(A,r1,r2,Zhi)
    while True: #emulate repeat/until
        Z=(Zlo+Zhi)/2.0
        tn=FindTTrial(A,r1,r2,Z)
        if (t-tn)*tnlo>0:
            Zlo=Z
        else:
            Zhi=Z
        if abs(Zlo-Zhi)<=eps:
            break
    
    Y=r1+r2-A*(1.0-Z*S(Z))/np.sqrt(C(Z))
    f=1.0-Y/r1
    g=A*np.sqrt(Y)
    gdot=1.0-Y/r2
    vv1=(rv2     -rv1*f)/g
    vv2=(rv2*gdot-rv1  )/g
    if l_DU is not None:
        vv1=su_to_cu(vv1,l_DU,mu,1,-1,inverse=True)
        vv2=su_to_cu(vv2,l_DU,mu,1,-1,inverse=True)
    return (vv1,vv2)

In [ ]:
def test_gauss1():
    #Canonical unit numbers from Vallado example 7-5, p467
    #and time between, and find velocities
    r0_cu=np.array([2.5,0.0,0.0])
    v0_cu_standard=np.array([0.2604450,0.3688589,0.0])
    r1_cu=np.array([1.915111,1.606969,0.0])
    v1_cu_standard=np.array([-0.4366104,0.1151515,0.0])
    dt=5.6519
    (v0_cu,v1_cu)=gauss(r0_cu,r1_cu,dt)
    print("test_gauss1() Calculated: ",v0_cu,v1_cu)
    print("test_gauss1() Documented: ",v0_cu_standard,v1_cu_standard)
    diff0=np.sqrt(np.sum((v0_cu-v0_cu_standard)**2))
    diff1=np.sqrt(np.sum((v1_cu-v1_cu_standard)**2))
    print("test_gauss1() difference: ",diff0,diff1)
    assert(diff0<3e-6 and diff1<3e-6)

In [ ]:
def test_gauss2():
    #Same numbers as from test_kepler, but we use begin and end positions
    #and time between, and find velocities
    r0_cu         =np.array([ 0.17738,-0.35784, 1.04614])
    v0_cu_standard=np.array([-0.71383, 0.54436, 0.30723])
    r1_cu         =np.array([-0.6616125, +0.6840739, -0.6206809])
    v1_cu_standard=np.array([ 0.4667380, -0.2424455, -0.7732126])
    dt=2.974674
    (v0_cu,v1_cu)=gauss(r0_cu,r1_cu,dt,Type=1)
    print("test_gauss2() Calculated: ",v0_cu,v1_cu)
    print("test_gauss2() Documented: ",v0_cu_standard,v1_cu_standard)
    diff0=np.sqrt(np.sum((v0_cu-v0_cu_standard)**2))
    diff1=np.sqrt(np.sum((v1_cu-v1_cu_standard)**2))
    print("test_gauss2() difference: ",diff0,diff1)
    assert diff0<3e-6 and diff1<3e-6

In [ ]:
test_gauss1()
test_gauss2()